In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchfly.modules.transformers import CachedBertEncoder, CachedBertDecoderLM, ChineseBERTBaseConfig
from torchfly.text.tokenizers import BertTokenizer
from torchfly.utils import get_pretrained_states
from torchfly.text.decode import top_filtering

ImportError: cannot import name 'top_filtering' from 'torchfly.text.decode' (/home/wuqy1203/Documents/Projects/TorchFly/torchfly/text/decode/__init__.py)

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [52]:
model_states = torch.load("models/model_state_epoch_2.th")

In [53]:
encoder = CachedBertEncoder(ChineseBERTBaseConfig)
decoder = CachedBertDecoderLM(ChineseBERTBaseConfig)

In [54]:
encoder.load_state_dict(model_states['encoder'], strict=False)
decoder.load_state_dict(model_states['decoder'], strict=False)

<All keys matched successfully>

In [55]:
device = torch.device("cuda")
encoder = encoder.to(device)
decoder = decoder.to(device)

In [56]:
val_data = torch.load("data/val_tokenized.pkl")

In [57]:
example = val_data[1]

In [58]:
def convert_tokens_to_tensor(tokens) -> torch.Tensor:
    tensor = [tokenizer._convert_token_to_id(token) for token in tokens]
    tensor = torch.LongTensor(tensor)
    return tensor

In [59]:
summary = ["[CLS]"] + example[0] + ["[SEP]"]
origin_headline = example[1] + ["[SEP]"]
expert_headline = ["[CLS]"] + example[2] + ["[SEP]"]

In [79]:
top_p = 0.9
temperature = 0.8
length = 0

# convert to torch Tensor
encoder_inputs = summary + origin_headline
encoder_inputs = convert_tokens_to_tensor(encoder_inputs)

encoder_inputs = encoder_inputs.unsqueeze(0).cuda()
mask = torch.ones_like(encoder_inputs).cuda()

with torch.no_grad():
    # the encoder
    _, past = encoder(encoder_inputs, mask=mask, past=None)

    start_tokens = convert_tokens_to_tensor(["[CLS]"]).unsqueeze(0).cuda()
    
    # cache saves in past
    mask = F.pad(mask, (0, 1), "constant", 1)
    
    logits, past = decoder(start_tokens, mask, past=past, past_length=0)
    logits = logits[:, -1, :] / temperature
    logits = top_filtering(logits, top_p=0.9)

    sentence = []

    probs = F.softmax(logits, dim=-1)
    prob, prev_pred = torch.topk(probs, k=1, dim=-1)
    sentence.append(prev_pred)
    length += 1
    
    # decoding loop
    for i in range(50):
        mask = F.pad(mask, (0, 1), "constant", 1.0)
        logits, past = decoder(prev_pred, mask, past=past, past_length=length)
        logits = logits.squeeze(1) / temperature
        logits = top_filtering(logits, top_p=0.9)
        probs = F.softmax(logits, dim=-1)
        prev_pred = torch.multinomial(probs, num_samples=1)
        sentence.append(prev_pred)
        length += 1
        
        if prev_pred.item() == 102:
            break

    sentence = torch.cat(sentence, dim=-1)

In [80]:
tokenizer.decode(sentence[0].tolist()).replace("##", "")

'这款八重洲的短波电台,到底是什么样子,你更期待哪个?[SEP]'

In [84]:
example[2]

['用', '17', '年', '前', '的', '配', '置', '还', '能', '打', '动', '消', '费', '者', '?']